In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import requests
import os
import datetime
import warnings
import pickle
import re
import seaborn as sns
from bs4 import BeautifulSoup
from time import sleep
warnings.filterwarnings('ignore')

In [2]:
team = pd.read_csv('team_base_advanced.csv', index_col = 0)
player = pd.read_csv('player_base_advance_game.csv', index_col = 0)

In [3]:
team['Home_Team_Won'] = team['W/L_Home'].map(lambda x: 1 if x == 'W' else 0)

In [5]:
player.groupby(['GAME_ID', 'TEAM_ID', 'TEAM'])['MIN'].sum()

GAME_ID   TEAM_ID     TEAM
21500001  1610612737  ATL     234
          1610612765  DET     215
21500002  1610612739  CLE     210
          1610612741  CHI     208
21500003  1610612740  NOP     208
                             ... 
22200322  1610612762  UTA     226
22200323  1610612754  IND     240
          1610612758  SAC     224
22200324  1610612747  LAL     241
          1610612757  POR     225
Name: MIN, Length: 17226, dtype: int64

In [21]:
check = player[player['GAME_ID'] == 22200324]
check[check['TEAM'] == 'LAL'].sort_values(by = 'MIN', ascending = False)

,GAME_ID,TEAM_ID,TEAM,PLAYER,MIN,MATCH_UP,GAME_DATE,W/L,PTS,FGM,...,AST_Ratio,OREB%,DREB%,REB%,TO_Ratio,eFG%,TS%,USG%,PACE,PIE
196,22200324,1610612747,LAL,Austin Reaves,35,LAL vs. POR,11/30/2022,W,22,7,...,17.6,5.4,8.3,6.8,0.0,80.0,81.4,16.7,98.29,15.7
198,22200324,1610612747,LAL,LeBron James,33,LAL vs. POR,11/30/2022,W,31,12,...,27.6,5.6,13.9,9.7,6.9,83.3,80.2,27.3,98.22,21.8
191,22200324,1610612747,LAL,Dennis Schroder,29,LAL vs. POR,11/30/2022,W,6,3,...,35.7,0.0,3.6,1.7,14.3,42.9,40.3,12.9,96.55,2.4
197,22200324,1610612747,LAL,Anthony Davis,29,LAL vs. POR,11/30/2022,W,27,9,...,4.3,15.2,22.6,18.8,8.7,55.9,64.4,30.6,98.03,21.0
194,22200324,1610612747,LAL,Russell Westbrook,23,LAL vs. POR,11/30/2022,W,10,4,...,21.1,4.2,15.4,10.0,0.0,35.7,33.6,25.4,104.80,6.1
193,22200324,1610612747,LAL,Patrick Beverley,22,LAL vs. POR,11/30/2022,W,5,2,...,0.0,0.0,19.0,8.7,0.0,35.7,35.7,13.5,94.15,2.6
195,22200324,1610612747,LAL,Thomas Bryant,20,LAL vs. POR,11/30/2022,W,14,6,...,25.0,5.3,23.8,15.0,0.0,72.2,74.2,18.8,108.31,14.6
189,22200324,1610612747,LAL,Max Christie,19,LAL vs. POR,11/30/2022,W,3,1,...,14.3,0.0,5.3,2.3,0.0,25.0,25.0,12.8,102.86,-1.3
190,22200324,1610612747,LAL,Kendrick Nunn,14,LAL vs. POR,11/30/2022,W,4,2,...,22.2,0.0,5.6,3.2,11.1,33.3,33.3,21.2,104.79,0.0
192,22200324,1610612747,LAL,Wenyen Gabriel,11,LAL vs. POR,11/30/2022,W,6,2,...,0.0,18.2,15.4,16.7,0.0,100.0,104.2,11.1,103.33,26.7


In [22]:
#!pip install basketball-reference-scraper
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
from basketball_reference_scraper.box_scores import get_box_scores

In [136]:
from nba_api.stats.endpoints import BoxScorePlayerTrackV2
#games_list = player['GAME_ID'].map(lambda x: '00' + str(x)).unique().tolist()
games_df = pd.read_csv('games_df.csv')
games_list = games_df['GAME_ID'].map(lambda x: '00' + str(x)).unique().tolist()

In [157]:
mins_df_1 = pd.DataFrame()

for game in games_list_1:

    df = BoxScorePlayerTrackV2(game_id = game).get_data_frames()[0]
    home = df['TEAM_ID'].unique()[1]
    away = df['TEAM_ID'].unique()[0]
    home_df = df[df['TEAM_ID'] == home][['GAME_ID', 'TEAM_ID', 'PLAYER_NAME', 'MIN']].reset_index(drop = True)
    away_df = df[df['TEAM_ID'] == away][['TEAM_ID', 'PLAYER_NAME', 'MIN']].reset_index(drop = True)
    joined = pd.concat([home_df, away_df], axis = 1, join = 'inner')
    mins_df_1 = pd.concat([mins_df_1, joined])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [137]:
games_list_2 = mins_df['GAME_ID'].unique().tolist()

In [155]:
games_list_1 = [i for i in games_list if i not in games_list_2]

In [139]:
len(games_list_1)

6481

In [146]:
games_list_3 = mins_df_1['GAME_ID'].unique().tolist()

In [149]:
df = pd.concat([mins_df, mins_df_1])

In [151]:
games_list_2 = df['GAME_ID'].unique().tolist()

In [156]:
len(games_list_1)

4113